Задача оценки стоимости автомобилей
есть тестовая выборка на соревновании
тренировчнуб надо собрать самой с сайта auto.ru

Помним, что по условию соревнования, нам нужно самостоятельно собрать обучающий датасет. В этом ноутбуке мы не будем рассматривать сбор данных. Предположим, что мы уже все собрали и просто подключили свой датасет через "Add Data", чтобы приступить к самому ML.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import sys
import pandas_profiling
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder

import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns

import re

import json

In [ ]:
print('Python       :', sys.version.split('\n')[0])
print('Numpy        :', np.__version__)

# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt
pd.set_option('display.max_columns', 50)

# фиксируем RANDOM_SEED, чтобы эксперименты были воспроизводимы
RANDOM_SEED = 42

In [ ]:
# функия оценки качества модели
def mape(y_true, y_pred):
    return np.mean(np.abs((y_pred-y_true)/y_true))

In [ ]:
# PATH_to_file_data = '/kaggle/input/all-brands-and-models-for-auto-ru-09-09-2020/'
# df_marks_and_models = pd.read_csv(PATH_to_file_data + 'all_car_brands_and_their_models_from_auto_ru.csv', index_col=0)
# print('Кол-во марок: ', df_marks_and_models.shape[0])
# display(df_marks_and_models.head(2))

# Setup

In [ ]:
VERSION    = 5
DIR_TRAIN  = '../input/auto-ru-900csv/'
DIR_TEST   = '../input/sf-dst-car-price-prediction/'
VAL_SIZE   = 0.20   # 20%

# Data

In [ ]:
!ls '../input'

In [ ]:
# train = pd.read_csv(DIR_TRAIN+'auto_ru_900.csv', sep=';') # датасет для обучения модели
# test = pd.read_csv(DIR_TEST+'test.csv')
# sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

Будем отдельно обрабатывать данные тренировочные и для итоговой оценки для сабмита.
Тренировочный сет парсился с учетом того, что я увидела и решила использовать из тестовых данных

In [ ]:
df = pd.read_csv(DIR_TRAIN+'auto_ru_900.csv', sep=';') # датасет для обучения модели



In [ ]:
df = df[~(df.vendor=='RUSSIAN')]

print('del russian cars ', len(df))

df.drop_duplicates(inplace=True)
print('del duplicates   ', len(df))

Смотрим на сами данные

In [ ]:
df.info()

**bodyType**

In [ ]:
df[df['bodyType'].isna()]

удаляем эти две записи - сильно много пропусков

In [ ]:
df = df[~(df['bodyType'].isna())]

In [ ]:
df.bodyType = df.bodyType.str.lower()
df.bodyType = df.bodyType.apply(lambda x : 'внедорожник' if 'внедорожник' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'седан' if 'седан' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'хэтчбек' if 'хэтчбек' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'универсал' if 'универсал' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'лифтбек' if 'лифтбек' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'купе' if 'купе' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'кабриолет' if 'кабриолет' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'тарга' if 'тарга' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'фургон' if 'фургон' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'родстер' if 'родстер' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'лимузин' if 'лимузин' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'минивэн' if 'минивэн' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'микровэн' if 'микровэн' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'компактвэн' if 'компактвэн' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'пикап двойная кабина' if 'пикап двойная кабина' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'пикап полуторная кабина' if 'пикап полуторная кабина' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'пикап одинарная кабина' if 'пикап одинарная кабина' in x else x)

In [ ]:
df.bodyType.value_counts()[:10]

In [ ]:
df.bodyType.value_counts()[10:20]

In [ ]:
da = df[(df.bodyType=='пикап двойная кабина') | (df.bodyType=='пикап полуторная кабина') | (df.bodyType=='пикап одинарная кабина')]
dven = df[(df.bodyType=='микровэн') | (df.bodyType=='минивэн') | (df.bodyType=='компактвэн')]

In [ ]:
sns.boxplot(x='bodyType',  y='price', data=da).set_title(f'boxplot пикапы' )

In [ ]:
sns.boxplot(x='bodyType',  y='price', data=dven).set_title(f'boxplot вэны' )

оставляю открытым вопрос стоит ли объединять к один тип пакипы (1,1.5, 2-ая кабины) и в один тип -вэны

**color**

In [ ]:
color_dict = {'040001': 'черный', 'FAFBFB':'белый', '97948F':'серый', 'CACECB':'серебристый',
             '0000CC':'синий', 'EE1D19':'красный','200204':'коричневый', '007F00':'зеленый', 
              'C49648':'бежевый', '22A0F8':'голубой', 
              'FFD600':'желтый', 'DEA522':'золотистый','FF8649':'оранжевый', '660099':'пурпурный',
              '4A2197':'фиолетовый', 'FFC0CB':'розовый'}

In [ ]:
df.color = df.color.replace(to_replace=color_dict)

In [ ]:
df.color.value_counts()[10:20]

In [ ]:
df.color = df.color.apply(lambda x: x if x in ['черный', 'белый', 
                                               'серый', 'синий', 'серебристый',
                                               'красный', 'коричневый', 'зеленый', 'оранжевый'] else 'другой')

In [ ]:
df.color.value_counts()

часть цветов объединили в категорию другой

**fuelType**

In [ ]:
df.fuelType.value_counts()

In [ ]:
fuel_type_dict = {'бензин':1, 'дизель':2, 'гибрид':3, 'электро':4, 'газ':5}
df.fuelType = df.fuelType.replace(to_replace=fuel_type_dict)

In [ ]:
df.fuelType.value_counts()

**numberOfDoors**

In [ ]:
df['numberOfDoors'] = df['numberOfDoors'].values.astype('int64')

In [ ]:
df.numberOfDoors.value_counts()

**productionDate**

In [ ]:
df.productionDate.value_counts()

**yearsOld**

In [ ]:
df['yearsOld'] = datetime.now().year - df.productionDate

In [ ]:
df.yearsOld.value_counts()

**vehicleTransmission**

In [ ]:
df.vehicleTransmission.value_counts()

In [ ]:
transmission_dict = {'автоматическая':1, 'AUTOMATIC':1,  'механическая':2, 'MECHANICAL':2, 
                     'вариатор':3, 'VARIATOR':3, 'роботизированная':4, 'ROBOT':4} 
df.vehicleTransmission = df.vehicleTransmission.replace(to_replace=transmission_dict)

In [ ]:
df.vehicleTransmission.value_counts()

**enginePower**

In [ ]:
df.enginePower.value_counts()

**mileage**

**Привод**

In [ ]:
df['Привод'].value_counts()

In [ ]:
wheeldrive_dict = {'передний':1, 'полный':2, 'задний':3}
df['Привод'] = df['Привод'].replace(to_replace=wheeldrive_dict)

In [ ]:
df['Привод'].value_counts()

**Руль**

In [ ]:
df['Руль'].value_counts()

In [ ]:
st_wheel_dict = {'Левый':1, 'Правый':0, 'LEFT':1, 'RIGHT':0}
df['Руль'] = df['Руль'].replace(to_replace=st_wheel_dict)

In [ ]:
df['Руль'].value_counts()

**Владельцы**

пропуски заполним значением 1 - 1 владелец

In [ ]:
df['Владельцы'].value_counts()

In [ ]:
df['Владельцы'].fillna(1, inplace=True)

признак owners - будет одинковым для трейна и теста


In [ ]:
owners_dict_train = {4.0:3, None:1}
df['owners'] = df['Владельцы'].replace(to_replace=owners_dict_train)
df['owners'] = df['owners'].values.astype('int64')



In [ ]:
df['owners'].value_counts()

**ПТС**

In [ ]:
df['ПТС'].value_counts()

In [ ]:
pts_dict = {'Оригинал':1, 'Дубликат':0, 'ORIGINAL':1, 'DUPLICATE':0}
df['ПТС'] = df['ПТС'].replace(to_replace=pts_dict)
df['ПТС'].fillna(1, inplace=True)

In [ ]:
df['ПТС'] = df['ПТС'].values.astype('int64')

In [ ]:
df['ПТС'].value_counts()

**Таможня**

не информативен - удалять

In [ ]:
df['Таможня'].value_counts()

**modelDate**

In [ ]:
df.modelDate.value_counts()

In [ ]:
df['modelDate'] = df['modelDate'].values.astype('int64')

**years_per_owner, years_owned**

[+] years_per_owner - сколько лет всреднем владел машиной каждый из владельцев

[+] years_owned - колько лет фактически владел машиной текущий владелец (если данные пропущены - заменяется на значение years_per_owner)


In [ ]:
def years_owned_f_train(y, m, correct_val):
    if str(y) == 'nan':
        return correct_val
    rez = int(datetime.now().year)-int(y) -1
    if int(m)==1:
        rez +=1
    return rez
    

In [ ]:
df['years_per_owner'] = round(df['yearsOld'] / df['owners'], 0)

df['years_owned'] = df.apply(lambda x: years_owned_f_train(x['Владение_year'], 
                                                          x['Владение_month'], x.years_per_owner), axis=1)

In [ ]:
df['years_per_owner'].isna().sum()

In [ ]:
df['years_owned'].min(), df['years_owned'].max()

In [ ]:
df[df.years_owned==98]

**start_date, hidden на удаление**

In [ ]:
df.start_date.nunique()

In [ ]:
df.hidden.value_counts()

**fuel_rate, acceleration, clearance**

[+] fuel_rate, acceleration, clearance - расход топлива в л, разгон до 100км/ч (с), клиренс - технические данные, полученные из признака "super_gen", пропуски заполнены следующим образом:

для каждой модели авто находилось среднее значение, им заполнялся соответствующий пропуск. если таких данных для модели авто не было - бралось общее среднее значение.


Если при обработке признака надо использовать другой признак - то вот так:
#df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)



In [ ]:
df.acceleration.value_counts()

In [ ]:
df.clearance.isna().sum()

In [ ]:
'''метод для применения в функции apply ко всему датафрейму
col1 - целевой признак, в котором заменяем пропуски,
df_ - данные, где записаны средние значения признака, сгруппированные по признаку bycol2

в данном случае col1 бцудет принимать значения fuel_rate, acceleration, clearance
bycol2 - будет принимать значение model - среднее считатем для каждой модели авто
df_ - будет передаваться список со средними значениями fuel_rate, 
acceleration, clearance для моделей машин'''

def col1_mean_by_col2_f(col1, df_, bycol2):
    if str(col1)=='nan':
        try:
            return df_[bycol2]
        except:
            return col1
    return col1

In [ ]:
df_clearance_by_model = df.groupby('model')['clearance'].mean()
df['clearance'] = df.apply(lambda x: col1_mean_by_col2_f(x['clearance'],
                                                         df_clearance_by_model, x['model']), axis=1)
df['clearance'].fillna(df['clearance'].mean(), inplace=True)


df_acceleration_by_model = df.groupby('model')['acceleration'].mean()
df['acceleration'] = df.apply(lambda x: col1_mean_by_col2_f(x['acceleration'],
                                                            df_acceleration_by_model, x['model']), axis=1)
df['acceleration'].fillna(df['acceleration'].mean(), inplace=True)


df_fuel_rate_by_model = df.groupby('model')['fuel_rate'].mean()
df['fuel_rate'] = df.apply(lambda x: col1_mean_by_col2_f(x['fuel_rate'], 
                                                         df_fuel_rate_by_model, x['model']), axis=1)
df['fuel_rate'].fillna(df['fuel_rate'].mean(), inplace=True)

**vendor**

In [ ]:
df.vendor.value_counts()

In [ ]:
vendor_dict = {'EUROPEAN':1, 'JAPANESE':2, 'KOREAN':3, 'AMERICAN':4, 'CHINESE':5, 'VENDOR_UNKNOWN':6}
df.vendor = df.vendor.replace(to_replace=vendor_dict)

In [ ]:
df.vendor.value_counts()

**modelDate**

In [ ]:
df.modelDate.value_counts()

In [ ]:
df['modelDate'] = df['modelDate'].values.astype('int64')

**delta_prod_modelYears**

[+] delta_prod_modelYears - разница между годом выпуска и годом выхода этой модели машины

In [ ]:
df['delta_prod_modelYears'] = df.productionDate-df.modelDate

**engineDisplacement**

все пропуски в этом признаке - в машинах электро-карах, заменяем значение на 0.0 (в виде стринга, это понадобится для последующей обработки данных, чтоб регулярка отработала

In [ ]:
df[(df.engineDisplacement.isna()) & (df.fuelType==4)]
df[df.engineDisplacement.isna()]

In [ ]:
df.engineDisplacement.fillna('0.0', inplace=True)

In [ ]:
df.engineDisplacement.unique()

In [ ]:
df.engineDisplacement = df.engineDisplacement.apply(lambda x:  float(re.findall("\d\.\d+", str(x))[0]))

In [ ]:
df.engineDisplacement.unique()

# Отбираем признаки

In [ ]:
num_cols = ['engineDisplacement', 'enginePower', 'numberOfDoors', 
            'delta_prod_modelYears','productionDate', 'owners', 'yearsOld', 'mileage', 
            'fuel_rate', 'acceleration', 'clearance', 'years_per_owner', 
            'years_owned']

cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'vehicleTransmission','ПТС', 
            'Привод' , 'Руль', 'vendor']

to_del = ['model', 'saleId', 'modelDate' , 'state_not_beaten', 'hidden', 'start_date', 'Владение_year',
         'Владение_month', 'Владение', 'Владельцы', 'Таможня']

In [ ]:
df.drop(columns=to_del, inplace=True)

In [ ]:
df.info()

**Собрали данные для обучения**

In [ ]:
data_ = df[num_cols+cat_cols+['price']].copy()

In [ ]:
data_.info()

In [ ]:
def has_outliers(column, df):

    q25, q75 = np.percentile(df[column], [25, 75], axis=0)
    iqr = q75-q25
    lower_bound = q25 - 1.5*iqr
    upper_bound = q75 + 1.5*iqr
    
#     print('---------------------------')
#     print('Границы выбросов: ', lower_bound, ', ', upper_bound)
#     print('---------------------------')

    ok_values_cnt = (df[column].loc[df[column].between(
        lower_bound, upper_bound)]).count()
    outliers_cnt = df[column].count() - ok_values_cnt

    print('Количество выбросов', outliers_cnt)
    print('---------------------------')
    
def boxplot_for_numcols(df, col_name, hue_col=None):
    plt.figure()
    # sns.boxplot(x=df[hue_col], y=np.log1p(np.abs(df[col_name])))
    if (hue_col != None):
        sns.boxplot(x=df[hue_col], y=df[col_name]).set_title(f'boxplot {col_name}')
        plt.show()
        sns.boxplot(x=df[hue_col], y=np.log1p(np.abs(df[col_name]))).set_title(f'boxplot LOG{col_name}')
        plt.show()


    else:
        sns.boxplot( y=df[col_name]).set_title(f'boxplot {col_name}')
        plt.show()
        sns.boxplot( y=np.log1p(np.abs(df[col_name]))).set_title(f'boxplot LOG{col_name}')
        plt.show()



    

def lookup_num_col(data, col_name):
    print('Количество пропусков: ', data[col_name].isna().sum())
    plt.figure(figsize=(6,4))
    
    sns.distplot(data[col_name], kde = False, rug=False).set_title(f'Распределение {col_name}')
    
    plt.show()
    sns.distplot(np.log1p(data[col_name]), kde = False, rug=False).set_title(f'Распределение LOG {col_name}')
    plt.show()
    has_outliers(col_name, data)
    boxplot_for_numcols(data, col_name)
    
    
def lookup_cat_col(data, col_name):
    print ('Количество пропусков: ', data[col_name].isna().sum())
    print ('Количество уникальных записей: ', data[col_name].nunique())
    if (data[col_name].nunique()<10):
        display(data[col_namee].value_counts())
    

    
    


# обработка тестовой выборки,чтоб имели одинаковый вид с тренировочной

In [ ]:
df = pd.read_csv(DIR_TEST+'test.csv')

Нижe обработка признаков валидационной выборки:

enginePower - сделала числовой признак, удалив текст

[+] delta_prod_modelYears - разница между годом выпуска и годом выхода этой модели машины

[+] yearsOld - 'возраст' машины (текущий год - год выпуска)

[+] fuel_rate, acceleration, clearance - расход топлива в л, разгон до 100км/ч (с), клиренс - технические данные, полученные из признака "super_gen", пропуски заполнены следующим образом:

для каждой модели авто находилось среднее значение, им заполнялся соответствующий пропуск. если таких данных для модели авто не было - бралось общее среднее значение.


[+] owners - к-тво владельцев 1,2,3 - 1,2,3 и больше соответственно

[+] years_per_owner - сколько лет всреднем владел машиной каждый из владельцев

[+] years_owned - колько лет фактически владел машиной текущий владелец (если данные пропущены - заменяется на значение years_per_owner)

In [ ]:
def years_owned_f(s, correct_val):
    if str(s)=='nan':
        return correct_val
    nwords = len(s.split())
    if nwords>2:
        return int(s.split()[0])
    if 'ме' in s:
        return 0
    return int(s.split()[0])



def parse_dict_f(str_as_dict, dict_param):
    try:
        d = json.loads(str_as_dict)[dict_param]
        return d
    except:
        return

#num
df.enginePower = df.enginePower.str.replace(' N12', '')
df.enginePower = df.enginePower.apply(lambda x: float(x))

df.engineDisplacement = df.engineDisplacement.str.replace(' LTR', '')
df.engineDisplacement = df.engineDisplacement.apply(lambda x: '0' if x=='' else x)
df.engineDisplacement = df.engineDisplacement.apply(lambda x: float(x))

df['delta_prod_modelYears'] = df.productionDate-df.modelDate
df['yearsOld'] = datetime.now().year - df.productionDate

df['fuel_rate'] = df['super_gen'].apply(lambda x: parse_dict_f(x, 'fuel_rate'))
df['acceleration'] = df['super_gen'].apply(lambda x: parse_dict_f(x, 'acceleration'))
df['clearance'] = df['super_gen'].apply(lambda x: parse_dict_f(x, 'clearance_min'))


# заполняем пропуски fuel_rate сначала средним значением для модели машины
# если пропуски остались - дозаполяем средним общим значением
#df['col_3'] = df.apply(lambda x: f(x.col_1, x.col_2), axis=1)

# def col1_mean_by_col2_f(col1, df_, bycol2):
#     if str(col1)=='nan':
#         try:
#             return df_[bycol2]
#         except:
#             return col1
#     return col1
    
# df_fuelrate_by_model = df.groupby('model_name')['fuel_rate'].mean()
# df['fuel_rate'] = df.apply(lambda x: col1_mean_by_col2_f(x['fuel_rate'], df_fuelrate_by_model, x['model_name']), axis=1)
# df['fuel_rate'].fillna(df['fuel_rate'].mean(), inplace=True)

df_clearance_by_model = df.groupby('model_name')['clearance'].mean()
df['clearance'] = df.apply(lambda x: col1_mean_by_col2_f(x['clearance'],
                                                         df_clearance_by_model, x['model_name']), axis=1)
df['clearance'].fillna(df['clearance'].mean(), inplace=True)


df_acceleration_by_model = df.groupby('model_name')['acceleration'].mean()
df['acceleration'] = df.apply(lambda x: col1_mean_by_col2_f(x['acceleration'],
                                                            df_acceleration_by_model, x['model_name']), axis=1)
df['acceleration'].fillna(df['acceleration'].mean(), inplace=True)


df_fuel_rate_by_model = df.groupby('model_name')['fuel_rate'].mean()
df['fuel_rate'] = df.apply(lambda x: col1_mean_by_col2_f(x['fuel_rate'], 
                                                         df_fuel_rate_by_model, x['model_name']), axis=1)
df['fuel_rate'].fillna(df['fuel_rate'].mean(), inplace=True)


#categories
owners_dict = {'1\xa0владелец': 1, '2\xa0владельца': 2, '3 или более': 3, None : 0}
df['owners'] = df['Владельцы'].replace(to_replace=owners_dict)

#num
df['years_per_owner'] = round(df['yearsOld'] / df['owners'],0)

df['years_owned'] = df.apply(lambda x: years_owned_f(x['Владение'], x.years_per_owner), axis=1)


# categories

df.color = df.color.apply(lambda x: x if x in ['черный', 'белый', 
                                               'серый', 'синий', 'серебристый',
                                               'красный', 'коричневый', 
                                               'зеленый', 'оранжевый'] else 'другой')

# custom_house_dict = {'Растаможен':1, 'Не растаможен': 0}
# df['Таможня'] = df['Таможня'].replace(to_replace=custom_house_dict)

# fuel_type_dict = {'бензин':1, 'дизель':2, 'гибрид':3, 'электро':4, 'газ':5}
df.fuelType = df.fuelType.replace(to_replace=fuel_type_dict)


# transmission_dict = {'автоматическая':1, 'механическая':2, 'вариатор':3, 'роботизированная':4} 
df.vehicleTransmission = df.vehicleTransmission.replace(to_replace=transmission_dict)

# vendor_dict = {'EUROPEAN':1, 'JAPANESE':2}
df.vendor = df.vendor.replace(to_replace=vendor_dict)

# pts_dict = {'Оригинал':1, 'Дубликат':0}
df['ПТС'] = df['ПТС'].replace(to_replace=pts_dict)
df['ПТС'] = df['ПТС'].values.astype('int64')

# wheeldrive_dict = {'передний':1, 'полный':2, 'задний':3}
df['Привод'] = df['Привод'].replace(to_replace=wheeldrive_dict)

# st_wheel_dict = {'Левый':1, 'Правый':0}
df['Руль'] = df['Руль'].replace(to_replace=st_wheel_dict)

# df.bodyType = df.bodyType.apply(lambda x : 'седан' if x=='седан 2 дв.' else x)
# df.bodyType = df.bodyType.apply(lambda x : 'хэтчбек' if x=='хэтчбек 3 дв.' else x)
# df.bodyType = df.bodyType.apply(lambda x : 'хэтчбек' if x=='хэтчбек 5 дв.' else x)
# df.bodyType = df.bodyType.apply(lambda x : 'внедорожник' if x=='внедорожник 5 дв.' else x)
# df.bodyType = df.bodyType.apply(lambda x : 'внедорожник' if x=='внедорожник 3 дв.' else x)

df.bodyType = df.bodyType.str.lower()
df.bodyType = df.bodyType.apply(lambda x : 'внедорожник' if 'внедорожник' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'седан' if 'седан' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'хэтчбек' if 'хэтчбек' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'универсал' if 'универсал' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'лифтбек' if 'лифтбек' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'купе' if 'купе' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'кабриолет' if 'кабриолет' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'тарга' if 'тарга' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'фургон' if 'фургон' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'родстер' if 'родстер' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'лимузин' if 'лимузин' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'минивэн' if 'минивэн' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'микровэн' if 'микровэн' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'компактвэн' if 'компактвэн' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'пикап двойная кабина' if 'пикап двойная кабина' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'пикап полуторная кабина' if 'пикап полуторная кабина' in x else x)
df.bodyType = df.bodyType.apply(lambda x : 'пикап одинарная кабина' if 'пикап одинарная кабина' in x else x)





#dates
df['parsing_date'] = df.parsing_unixtime.apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d'))



In [ ]:
df.info()

In [ ]:
# num_cols = ['engineDisplacement', 'enginePower', 'numberOfDoors', 
#             'delta_prod_modelYears', 'owners', 'yearsOld', 'mileage', 
#             'fuel_rate', 'acceleration', 'clearance', 'years_per_owner', 
#             'years_owned']

# cat_cols = ['brand', 'color', 'fuelType', 'vehicleTransmission','ПТС', 
#             'Привод' , 'Руль', 'vendor']

# to_del = ['model', 'saleId', 'modelDate' , 'state_not_beaten', 'hidden', 'start_date', 'Владение_year',
#          'Владение_month', 'Владение', 'Владельцы', 'Таможня']

In [ ]:
num_cols = ['engineDisplacement', 'enginePower', 'numberOfDoors', 
            'delta_prod_modelYears','productionDate', 'owners', 'yearsOld', 'mileage', 
            'fuel_rate', 'acceleration', 'clearance', 'years_per_owner', 
            'years_owned']

cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'vehicleTransmission','ПТС', 
            'Привод' , 'Руль', 'vendor']

to_del = ['car_url', 'complectation_dict', 'description', 'equipment_dict', 
          'image', 'model_info', 'model_name', 'super_gen', 'name', 
          'priceCurrency', 'vehicleConfiguration' , 'modelDate', 'sell_id', 'Владение', 
          'Владельцы', 'Таможня' , 'Состояние']

In [ ]:
df.drop(columns=to_del, inplace=True)
df.info()

**собрали данные для тестовой выборки на сабмит**

In [ ]:
test_ = df[num_cols+cat_cols].copy()

In [ ]:
df.parsing_date.unique()

In [ ]:
test_.info()

In [ ]:
data_.info()

In [ ]:
pandas_profiling.ProfileReport(data_)

Кратко про увиденное в отчете:
нули в переменных не смущают, так и должно быть - количество лет владения машиной, пробег, и другие характеристики могут быть нулевыми.

Признак brand имеет большое количество значений, возможно пригодится доп обработка.

Из heat-map коррлеции пирсена видно, что прямо коррелируют мощность и объем двигателя, но для электрокаров это будет не справедливо, просто избавиться от одного из признаков не видится разумным. Так же возраст машины, пробег, срок владения коррелируют.
Целевой признак Цена - прямо коррелирует с мощностью двигателя (enginePower), обратно-корр с разгоном до 100км, что выглядит логично.

если рассмотреть корреляцию Спирмена, то видно, то цена дял старых, с пробегом машин будет ниже, у мощных по характеристикам двигателя и новым по году выпуска машин цена будет выше. Никаких аномальных наблюдений не видно в других подсчетах корреляций.

In [ ]:
pandas_profiling.ProfileReport(test_)

**Если добавлять новые признаки и смотреть на их importance**

In [ ]:
# # добавим новые признаки colA_colB = colA*colB
# columns = X.columns
# for col1 in columns:
#     for col2 in columns:
#         if col2 != col1 and col1+'_'+col2 not in X.columns and col2+'_'+col1 not in X.columns:
#             X[col1+'_'+col2] = X[col1] * X[col2]


# gbc = GradientBoostingClassifier(random_state=42)
# importances_S = pd.Series(gbc.feature_importances_,
#                           X.columns).sort_values(ascending=False)
# importances_S.plot(kind='bar', title="Feature Importances")

## Data Preprocessing

 ## Label Encoding
 
 data_ содержит тренировочный сет
 
 test_ содержит сет для сабмита

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
data_['sample'] = 1 # помечаем где у нас трейн
test_['sample'] = 0 # помечаем где у нас тест

data = test_.append(data_, sort=False).reset_index(drop=True) # объединяем

In [ ]:

for colum in ['bodyType', 'brand', 'color']:
    data[colum] = data[colum].astype('category').cat.codes

In [ ]:
data.info()

In [ ]:
X = data.query('sample == 1').drop(['sample', 'price'], axis=1)
y = data.query('sample == 1').price
X_sub = data.query('sample == 0').drop(['sample', 'price'], axis=1)

## Train Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=VAL_SIZE, 
                                                    shuffle=True, random_state=RANDOM_SEED)

# # Model : CatBoost
![](https://pbs.twimg.com/media/DP-jUCyXcAArRTo.png:large)   


У нас в данных практически все признаки категориальные. Специально для работы с такими данными была создана очень удобная библиотека CatBoost от Яндекса. [https://catboost.ai](http://)     
На данный момент **CatBoost является одной из лучших библиотек для табличных данных!**

#### Полезные видео о CatBoost (на русском):
* [Доклад про CatBoost](https://youtu.be/9ZrfErvm97M)
* [Свежий Туториал от команды CatBoost (практическая часть)](https://youtu.be/wQt4kgAOgV0) 

## Fit

In [ ]:
cat_cols

In [ ]:
data[cat_cols].info()

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, y_train,
         cat_features=cat_cols,
         eval_set=(X_test, y_test),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_baseline.model')

In [ ]:
# оцениваем точность
predict = model.predict(X_test)
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict))*100:0.2f}%")

Получили уровень ошибки 13.54%!

### Log Traget
Попробуем взять таргет в логорифм - это позволит уменьшить влияние выбросов на обучение модели (используем для этого np.log1p, np.expm1 (log(1+x) и exp(x)-1).    


In [ ]:
np.log(y_train)

In [ ]:
model = CatBoostRegressor(iterations = 5000,
                          random_seed = RANDOM_SEED,
                          eval_metric='MAPE',
                          custom_metric=['R2', 'MAE'],
                          silent=True,
                         )
model.fit(X_train, np.log1p(y_train),
         cat_features=cat_cols,
         eval_set=(X_test, np.log1p(y_test)),
         verbose_eval=0,
         use_best_model=True,
         #plot=True
         )

model.save_model('catboost_single_model_2_baseline.model')

In [ ]:
predict_test = np.expm1(model.predict(X_test))
predict_submission = np.expm1(model.predict(X_sub))

**корректировка курса доллара**

от времени сбора сабмит-данных, до времени сбора тренировочных данных курс доллара изменился. Возьмем очень грубо (и не очень красиво конечно), что средний курс октбяря был 76Rub, а срежний курс января 74Rub. Приниамем, что цена машины в доллараха постоянна, а в рублях ее обновляют периодически на сайте (именно такое и наблюдалось в спаршенной с сайта информации) 

In [ ]:
usd_rate = 76/74


pred_sub_usd = usd_rate*predict_submission
pred_sub_usd

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Как видим точность возросла до 11.3%, а что будет на leaderboard?

# Submission

In [ ]:
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')
# sample_submission['price'] = predict_submission

sample_submission['price'] = pred_sub_usd
sample_submission.to_csv(f'submission_2_v{VERSION}.csv', index=False)
sample_submission.head(10)

С корректировкой на курс доллара получили **MAPE 16.86%** на ЛБ

попробуем улучшить модель
в тренировочных данных так же имеется инфа про новые машины без проега, в тестовых же идет речь только про подержанные машины. Но я все ж думаю, что не это портит оценку.
Так что будем пробовать другие алгоритмы

Ниже закомментированный код - так как кагл очень долго будет пушить версию

этот код запускался и прерывался, так как по графику стало похоже, что минимальное значение получено на 104 итерации подбора параметров. там вышло :

depth=10
learning_rate = 0.062
l2_leaf_reg=1

In [ ]:
# cbr = CatBoostRegressor(random_seed = RANDOM_SEED,
#                           eval_metric='MAPE',
#                           custom_metric=['R2', 'MAE'],
#                           silent=True,)
# grid = {'learning_rate': [round(x,3) for x in 
#                           np.linspace(start = 0.025, stop = 0.2, num = 20)],
#         'depth': [int(x) for x in 
#                   np.linspace(start = 5, stop = 20, num = 4)],
#         'l2_leaf_reg': [int(x) for x in 
#                         np.linspace(start = 1, stop = 5, num = 5)]}

In [ ]:
# grid_search_result = cbr.grid_search(grid, 
#                                       X=X_train, 
#                                       y=np.log1p(y_train), 
#                                       plot=True)
# print(grid_search_result['params'])


![](https://i.ibb.co/n3GLNCz/photo-2021-01-24-08-29-30.jpg:large)

Подбор параметров для градиентногобустинга

In [ ]:
# from sklearn.ensemble import GradientBoostingRegressor
# gbr = GradientBoostingRegressor(random_state = RANDOM_SEED)

# from sklearn.model_selection import GridSearchCV
# grid_search = GridSearchCV(gbr,
#                            param_grid={'learning_rate': [ 0.05, 0.1, 1],
#                                        'n_estimators': [100,  350, 500, 750, 1000]},
#                            n_jobs=-1, cv=3
#                            )
# grid_search.fit(X_train, np.log1p(y_train))

# grid_search.best_params_



в результате получилось

{'learning_rate': 0.1, 'n_estimators': 1000}

Подбирать параметры - очень дорогое по времени удовольствие
В итоговый сабмит взяла параметры немного наугад (выходило, что чем больше n_estimators, тем дольше работает обучение, взяла меньше)

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor




estimators=[
        ('rf', RandomForestRegressor(random_state=RANDOM_SEED, 
                                      n_estimators=200, min_samples_split=2, 
                                      min_samples_leaf=2, 
                                      max_features='auto', max_depth=5, 
                                      bootstrap=True, n_jobs=-1)),
        ('gbr', GradientBoostingRegressor(n_estimators=1000, random_state=RANDOM_SEED,  
                                          )),
        ('сbr',  CatBoostRegressor(iterations = 2000,
                                   learning_rate = 0.062,
                                   random_seed = RANDOM_SEED,
                                   eval_metric='MAPE',
                                   custom_metric=['R2', 'MAE'],
                                   depth=10,
                                   l2_leaf_reg=1)
        )
            ]

sr = StackingRegressor(estimators=estimators,
                       final_estimator = GradientBoostingRegressor(n_estimators=500, 
                                                                   random_state=RANDOM_SEED))

In [ ]:
sr.fit(X_train, np.log1p(y_train))

In [ ]:
predict_test = np.expm1(sr.predict(X_test))
predict_submission = np.expm1(sr.predict(X_sub))

In [ ]:
print(f"Точность модели по метрике MAPE: {(mape(y_test, predict_test))*100:0.2f}%")

Точность модели с использованием стекинга стала 11.07%

In [ ]:
usd_rate = 76/74


pred_sub_usd = usd_rate*predict_submission
pred_sub_usd

будем использовать корректировку курса доллара 
и подготовим 2 сабмита


In [ ]:
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')
sample_submission['price'] = predict_submission

sample_submission.to_csv(f'submission_2_v{VERSION}_stack.csv', index=False)


sample_submission['price'] = pred_sub_usd
sample_submission.to_csv(f'submission_2_v{VERSION}_stack_usd.csv', index=False)
#sample_submission.head(10)

стекинг дал улучшение метрики МАРЕ на ЛБ до 15%
с учетом корректировки на курс доллара - получилось 16%

добавим еще сабмиты с округлением с точностью до 1000 рублей

In [ ]:
pr_sub_round = np.round(predict_submission/1000)*1000
pred_sub_usd_round = np.round(pred_sub_usd/1000)*1000

In [ ]:
sample_submission = pd.read_csv(DIR_TEST+'sample_submission.csv')

sample_submission['price'] = pr_sub_round

sample_submission.to_csv(f'submission_2_v{VERSION}_stack_round.csv', index=False)


sample_submission['price'] = pred_sub_usd_round
sample_submission.to_csv(f'submission_2_v{VERSION}_stack_usd_round.csv', index=False)

Подробный чек лист: https://docs.google.com/spreadsheets/d/1I_ErM3U0Cs7Rs1obyZbIEGtVn-H47pHNCi4xdDgUmXY/edit?usp=sharing

# Выводы

Опробовав стекинг - очень мощные вычисления, подбор параметров и сами вычислени требуют больших мощностно-временных затрат

скоринг, который получился на лидерборде при первом прогоне catboost с корректировкой на курс доллара дал 16% МАРЕ
добавила много вычислительного времени, стекинг - и получила 15%. (с корректировкой курса доллара получала те же 16%)
Стоит ли игра свеч? :)